In [16]:
#----Fonctions ayant permis la creation du fichier JSON de questions-réponses--------#

In [17]:
#recuperation du label d'une entité Wikidata en lang (la langue fournie en paramètre)
def requete(lang,param) :   
    requete = """
    SELECT DISTINCT (wd:%s AS ?reponse) ?reponseLabel WHERE
    {
      wd:%s rdfs:label ?reponseLabel.
      FILTER(lang(?reponseLabel)="%s").
    }
    """%(param,param,lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    reponse = requests.get(url, params={'query': requete, 'format': 'json'})
    if reponse.status_code == 200 :
        data = reponse.json()
    else :
        data = {}

    return data

In [18]:
#récupération de réponses candidates pour une entité Wikidata donnée grâce à Wembedder
#https://github.com/fnielsen/wembedder
def reponses_candidates(reponse,langues) :
    candidatsR = {}
    candidats = []
    url = 'https://wembedder.toolforge.org/api/most-similar/'+reponse
    response = requests.get(url, params={'query': requete, 'format': 'json'})
    if response.status_code == 200 :
        data = response.json()
        #je verifie que la reponse a bien des réponses similaires
        if data.get('most_similar')!= None :
            liste_sim = data['most_similar']
            for i,el in enumerate(liste_sim):
                if i == 3 :
                    break
                candidats.append(el['item'])

        #on recupere l'identifiant et le label des propositions 
        #pour chaque proposition
        for c in candidats :
            candidatsR[c] = {}
            #pour chaque langue
            for lang in langues :
                #on recupere le label
                dico_reponse = requete(lang,c)
                #print(dico_reponse)
                if len(dico_reponse['results']['bindings']) != 0 :
                    cle = dico_reponse['results']['bindings'][0]['reponse']['value'].split('http://www.wikidata.org/entity/')[1]
                    valeur = dico_reponse['results']['bindings'][0]['reponseLabel']['value']
                    candidatsR[c][lang] = valeur
      
            time.sleep(5)

    return candidatsR

In [19]:
#extraction de questions et de réponses du corpus d'entraînement de QA QALD-9-plus
#https://github.com/KGQA/QALD_9_plus
def extraction_qald(chemin,nom) :
    if os.path.exists(nom) :
        return nom
    liste_quiz = []
    dico_QA = ouvrir_json(chemin)
    #on souhaite enlever les questions larges (qui ne sont pas des interrogations)
    QA_list = dico_QA["questions"]
    #dico_questions = {}
    langues = ['fr','en']
    #cpt = 0
    for i,q in enumerate(QA_list) :
        #q["question"] : liste de dictionnaires
        quiz = {}
        quiz["question"] = {}
        quiz["reponse"] = {}
        quiz["candidats"] = {}
        
        quizValide = False
        estInvalide = False #cas où une reponse ou une proposition ne renvoie pas de resultat pour le label
        
        for question in q["question"] :
            #question : dictionnaire dont cle : language et cle : 'string' 
            for k1,v1 in question.items() :
                if k1 == 'language' and (v1 == 'en' or v1 == 'fr') :
                    quiz["question"][v1] = question['string']
        for rep in q["answers"] :
            for k2,v2 in rep.items() :
                if k2 == 'results' :
                    #on a besoin de la cle pour acceder a la valeur de la reponse 
                    cle = rep["head"]["vars"][0]
                    
                    #on verifie que la question n'a qu'une reponse (c'est le seul cas qui nous interesse)
                    #on filtre aussi les reponses sont bien des entites wikidata (on ne peut pas faire de resumes d'entites sur eux)
                    if len(v2['bindings']) == 1 and re.match(r'http://www.wikidata.org/entity/Q\d+',v2['bindings'][0][cle]['value']):
                        Ganswer = v2['bindings'][0][cle]['value'].split('http://www.wikidata.org/entity/')[1]
                        
                        quizValide = True
                        
        if quizValide :
            #on extrait le label de la reponse en anglais en français 
            quiz["reponse"][Ganswer] = {}
            #on fait la requete pour la reponse 
            for l in langues :
                dico_reponse = requete(l,Ganswer)
                print(dico_reponse)
                #dans le cas où la requete ne renvoie pas de resultat
                if len(dico_reponse['results']['bindings']) != 0 :
                    valeur = dico_reponse['results']['bindings'][0]['reponseLabel']['value']
                    quiz["reponse"][Ganswer][l] = valeur
                else :
                    estInvalide = True
            time.sleep(5.0)
            candidats = reponses_candidates(Ganswer,langues) 
            quiz["candidats"] = candidats
            liste_quiz.append(quiz)
            write_json(nom,liste_quiz)
    write_json(nom,liste_quiz)
    return liste_quiz

In [20]:
#-------Fin----------------------------------------------------------------------------------#

In [21]:
#importations
import requests
import random
from random import randrange
import json
import os
import glob
import re
import time

In [31]:
#fonction pour écrire dans un fichier json
def write_json(chemin,contenu):
    with open(chemin,"w",encoding="utf-8") as w :
        w.write(json.dumps(contenu, indent=2, ensure_ascii=False))

In [32]:
#fonction pour récupérer le contenu d'un fichier json
def ouvrir_json(chemin) :
    with open(chemin,encoding="utf-8") as f :
        contenu = json.load(f)
    return contenu

In [22]:
#https://github.com/athalhammer/danker
#enregistrement des identifiants Wikidata et de leur score dans un dictionnaire
#le fichier 2021-11-15.allwiki.links.rank étant trop lourd, il n'est pas dans le répertoire ressources mais est téléchargeable au lien :
#https://danker.s3.amazonaws.com/index.html
def dico_wikidata_ranks(chemin="ressources/2021-11-15.allwiki.links.rank") :
    dico_ranks = {}
    cpt = 0
    with open(chemin,"r",encoding="utf-8") as f :
        ligne = f.readline()
        #on recupere la ligne, en enlevant le saut de ligne
        ligne = ligne[:len(ligne)-1]
        #pour chaque ligne
        while ligne :
            cpt+=1
            #un fichier .ranks a ses donnees separees par une tabulation
            liste = ligne.split("\t")
            #on recupere les donnees dans un dictionnaire (identifiant_entite - score)
            dico_ranks[liste[0]] = float(liste[1])

            ligne = f.readline()
            ligne = ligne[:len(ligne)-1]

    return dico_ranks

In [23]:
#classement des proprietes d'une entité Wikidata selon PageRank
def ranking_properties(dico_ranks,dico_reponses) :
    notes = {}
    cpt = 0
    #classement des proprietes de la reponse
    for k,v in dico_reponses.items() :
        #si la valeur (de la propriété) est dans le dictionnaire de rangs
        if dico_ranks.get(k) != None :
            rang = dico_ranks[k]
            notes[rang] = k
            cpt+=1
   
    #sortie : {score : identifiant}
    notes_triees = sorted(notes.keys(),reverse=True)
    dico_trie = {}
    for n in notes_triees :
        dico_trie[n] = notes[n]
        
    return dico_trie

In [24]:
def choix_reponse(fichier,langue) :
    liste_quiz = ouvrir_json(fichier)
    dico_reponse = {}
    estValide = False
    cpt = 0
    while estValide == False :
        cpt +=1
        reponse_possible = random.choice(liste_quiz)
        #je recupere l'identifiant de la reponse
        rep = [ v.get(langue) for k,v in reponse_possible["reponse"].items()][0]
       
        if reponse_possible["question"].get(langue) != None and rep != None and len(reponse_possible["candidats"]) == 3 :
            estValide = True
            reponse = reponse_possible
        
    
    dico_reponse["question"] = reponse["question"][langue]
    #je recupere la clé (identifiant) de reponse,que je convertis en list (type manipulable) dont je récupère la 1ère (et seule) valeur
    identifiant_reponse = list(reponse["reponse"].keys())[0]
    valeur_reponse = reponse["reponse"][identifiant_reponse][langue]
    dico_reponse["reponse"] = {valeur_reponse : identifiant_reponse}
    
    dico_reponse["candidats"] = {}
    for k,v in reponse["candidats"].items() :
        dico_reponse["candidats"][v[langue]] = k
    
    return dico_reponse

In [25]:
def resume(lang,reponse,dico_rangs) :

    requete = """
        SELECT ?propLabel ?o ?oLabel
        WHERE {
          BIND (wd:%s AS ?s)
          ?s ?prop ?o .
          ?propriete wikibase:directClaim ?prop .
          ?propriete rdfs:label ?propLabel .
          ?o rdfs:label ?oLabel .
          FILTER (lang(?oLabel) = "%s")
          FILTER (lang(?propLabel) = "%s")
        } 

    """%(reponse,lang,lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    response = requests.get(url, params={'query': requete, 'format': 'json'})
    if response.status_code == 200 :
        data = response.json()
    else :
        data = {}

    dico_reponse = {}
    liste_niv1 = data["results"]["bindings"]
    for l in liste_niv1 :
        if re.search(r'^http://www.wikidata.org/entity/.*',l['o']['value']) :
            identifiant = l['o']['value'].split("http://www.wikidata.org/entity/")[1]
            if identifiant not in dico_reponse :
                dico_reponse[identifiant] = [(l['propLabel']['value'],l['oLabel']['value'])]
            else :
                dico_reponse[identifiant].append((l['propLabel']['value'],l['oLabel']['value']))
    #dictionnaire : score : identifiant_lexeme
    dico_rangs = ranking_properties(dico_rangs,dico_reponse)
    #print(dico_rangs)
    return dico_reponse,dico_rangs,reponse

In [26]:
def comparaison_entites(bad_answer,good_answer,lang,dico_rangs) :
    informations = {}
    limite = 7
    #pour deux entites, on cherche les prédicats et les objets communs
    requete = """
    SELECT DISTINCT ?p ?propLabel ?o ?oLabel  WHERE
    {

      wd:%s ?p ?o.
      wd:%s ?p ?o.
      ?prop wikibase:directClaim ?p .
      ?prop rdfs:label ?propLabel .
      FILTER (lang(?propLabel) = "%s")
      ?o rdfs:label ?oLabel .
      FILTER (lang(?oLabel) = "%s")
    }
    """%(bad_answer,good_answer,lang,lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    response = requests.get(url, params={'query': requete, 'format': 'json'})
    if response.status_code == 200 :
        data = response.json()
    else :
        data = {}

    dico_communs = {}
    liste_niv1 = data["results"]["bindings"]
    
    #dictionnaire : identifiant_entite : [propriete,valeur]
    for l in liste_niv1 :
        cle = l['o']['value'].split('http://www.wikidata.org/entity/')[1]
        dico_communs[cle] = [l['propLabel']['value'],l['oLabel']['value']]
        
    #trie les valeurs de dico_communs -> score : [propriete,valeur]
    rang_prop = ranking_properties(dico_rangs,dico_communs)
    
    i = 0

    for k,v in rang_prop.items() :
        if i == 7 :
            break
        cle = dico_communs[v][0]
        valeur = dico_communs[v][1]
        informations[cle] = valeur
        i+=1

        
    return informations

In [27]:
def preparation_reponse(dico_reponse,dico_rangs) :

    indices = {}
    limite = 6
    cpt = 0
   
    #ce qu'on veut faire : récupérer un seul prédicat (la redondance semble non pertinente)
    
    for k,v in dico_rangs.items() :
        rep = dico_reponse[v]
        #comme un prédicat peut etre lie a plusieurs objets, je choisis le premier objet
        cle = rep[0][0]
        valeur = rep[0][1]
        #je choisis l'objet le plus populaire
        if cle not in indices :
            indices[cle] = valeur
            cpt+=1
            if cpt == 7 :
                break

    return indices

In [28]:
def affichage_dictionnaires(dico) :
    #affichage dictionnaire de façon esthétique
    print("*"*100)
    for k,v in dico.items() :
        print("%s : %s"%(k,v))
    print("*"*100)
    return  

In [4]:
dico_pageRank = dico_wikidata_ranks()

In [29]:
def jeu(chemin,dico_pageRank) :
    print("Quiz !!")
    lang = input("Quelle langue choisissez-vous ? Anglais (en) ou Français (fr) ?")
    
    cpt = 0
    print("-"*100)
    while cpt < 5 :
        
        resultat = choix_reponse(chemin,lang)
        #print(reponse_dico)
        #on souhaite afficher les propositions a des emplacements differents
        p = [0,1,2,3]
        random.shuffle(p)
        
        print(resultat["question"])
        
        propositions = []
        reponseB = [k for k,v in resultat['reponse'].items()][0]
        propositions.append(reponseB)
        for k,v in resultat['candidats'].items() :
            propositions.append(k)
        
        aff_propositions = []
        for i in p :
            aff_propositions.append(propositions[i])
        for el in aff_propositions :
            print(el)
        
        reponse_user = input("Quelle est votre réponse ?")
        
        if reponse_user.lower() == reponseB.lower() :
            print()
            print("Bravo !!")
            print()
            #on recupere le resume
            
            dico_reponse,dico_ranks_entity,reponse = resume(lang,resultat['reponse'][reponseB],dico_pageRank)
            #on le prepare
            if len(dico_reponse) > 0 :
                infos = preparation_reponse(dico_reponse,dico_ranks_entity)
                #affichage esthetique
                affichage_dictionnaires(infos)
        else :
            print()
            print("Non ! La bonne réponse est : ",reponseB)
            print()
            #donnees sur la bonne reponse
            dico_reponse,dico_ranks_entity,reponse = resume(lang,resultat['reponse'][reponseB],dico_pageRank)
            #dans le cas où data ne renvoie pas un json (eviter le json decode error)
            if len(dico_reponse) > 0 :
                #on le prepare 
                infos = preparation_reponse(dico_reponse,dico_ranks_entity)
                print("Voici des informations sur la réponse")
                affichage_dictionnaires(infos)
            
            #on recupere l'identifiant de la mauvaise reponse
            mauvaise_rep = resultat['candidats'][reponse_user]
            #appel fonction comparaison entite
            dico_commun = comparaison_entites(mauvaise_rep,resultat['reponse'][reponseB],lang,dico_pageRank)
            if len(dico_commun) > 0 :
                print()
                print("Voici les points communs entre votre réponse et la bonne réponse : ")
                affichage_dictionnaires(dico_commun)
            else : 
                print()
                print("Nous n'avons pas trouvé de point commun")
        print("-"*100)
        cpt+=1
    return

In [33]:
jeu("ressources/fichier_quiz_en_fr.json",dico_pageRank)

Quiz !!
Quelle langue choisissez-vous ? Anglais (en) ou Français (fr) ?fr
----------------------------------------------------------------------------------------------------
A quelle espèce appartient l'éléphant ?
amphibien
mammifères
animal
Lepidoptera
Quelle est votre réponse ?mammifères

Bravo !!

****************************************************************************************************
rang taxinomique : classe
nature de l'élément : taxon
décrit par : Dictionnaire encyclopédique Brockhaus et Efron
ce taxon produit : lait
taxon supérieur : Tetrapoda
discipline dont c'est l'objet : mammalogie
catégorie : Catégorie:Mammifère
****************************************************************************************************
----------------------------------------------------------------------------------------------------
Quelle est la plus grande ville d'Australie?
Melbourne
Brisbane
Adélaïde
Sydney
Quelle est votre réponse ?Sydney

Bravo !!

*****************************